In [185]:
import pandas as pd
from tqdm import tqdm
import requests
import os
import plotly.express as px

In [186]:
file_name = "proteins-2025-01-11.csv"

In [187]:
df = pd.read_csv(file_name)

In [188]:
df.shape

(8915, 33)

In [189]:
df.head()

,id,ordering,family_name_cache,species_name_cache,membrane_name_cache,name,description,comments,pdbid,resolution,...,superfamily_id,classtype_id,type_id,secondary_representations_count,structure_subunits_count,citations_count,created_at,updated_at,uniprotcode,interpro
0,1,6024.0,OmpA family,Escherichia coli,Gram-neg. outer,"Outer membrane protein A (OMPA), disordered loops",NaN,OmpA is required for the action of colicins K ...,"=""1qjp""",1.65,...,26,2,1,3,1,2,2018-08-13 03:49:46 UTC,2023-02-22 20:43:56 UTC,OMPA_ECOLI,NaN
1,2,6027.0,Enterobacterial Ail/Lom protein,Escherichia coli,Gram-neg. outer,Outer membrane protein X (OMPX),NaN,OmpX from Escherichia coli promotes adhesion t...,"=""1qj8""",1.9,...,26,2,1,7,1,1,2018-08-13 03:49:46 UTC,2023-02-22 20:43:57 UTC,OMPX_ECOLI,NaN
2,3,6032.0,Opacity porins,Neisseria meningitidis,Gram-neg. outer,Outer membrane protein NspA,NaN,Pathogenic Neisseria spp. possess a repertoire...,"=""1p4t""",2.55,...,235,2,1,0,1,0,2018-08-13 03:49:46 UTC,2023-02-22 20:43:57 UTC,Q9RP17_NEIME,NaN
3,4,5624.0,Influenza virus matrix protein 2,Influenza A virus,Viral,"M2 proton channel of Influenza A, closed state...",NaN,NaN,"=""3lbw""",1.65,...,185,11,1,3,4,0,2018-08-13 03:49:46 UTC,2023-02-22 20:43:54 UTC,M2_I97A1,NaN
4,5,6047.0,"OM protease omptin, OMPT",Yersinia pestis,Gram-neg. outer,Plasminogen activator PLA (coagulase/fibrinoly...,NaN,NaN,"=""2x55""",1.85,...,27,2,1,2,1,0,2018-08-13 03:49:46 UTC,2023-02-22 20:43:56 UTC,COLY_YERPE,NaN


In [190]:
df.columns

Index(['id', 'ordering', 'family_name_cache', 'species_name_cache',
       'membrane_name_cache', 'name', 'description', 'comments', 'pdbid',
       'resolution', 'topology_subunit', 'topology_show_in', 'thickness',
       'thicknesserror', 'subunit_segments', 'tilt', 'tilterror', 'gibbs',
       'tau', 'verification', 'membrane_id', 'species_id', 'family_id',
       'superfamily_id', 'classtype_id', 'type_id',
       'secondary_representations_count', 'structure_subunits_count',
       'citations_count', 'created_at', 'updated_at', 'uniprotcode',
       'interpro'],
      dtype='object')

In [191]:
def get_sequences_from_uniprot(uniprot_ids):
    """
    Retrieves protein sequences from UniProt for the given list of UniProt IDs.

    Args:
        uniprot_ids (list): List of UniProt IDs (e.g., ['P12345', 'Q67890']).

    Returns:
        dict: A dictionary where keys are UniProt IDs and values are
        protein sequences.
    """
    base_url = "https://rest.uniprot.org/uniprotkb"
    sequences = {}

    for uniprot_id in tqdm(
        uniprot_ids, desc="Fetching UniProt Sequences", unit="protein"
    ):
        # Construct the URL for the FASTA request
        url = f"{base_url}/{uniprot_id}.fasta"
        response = requests.get(url)

        # Check if the request was successful
        if response.status_code == 200:
            fasta_content = response.text
            # Extract the sequence from the FASTA content
            sequence = "".join(
                fasta_content.splitlines()[1:]
            )  # Skip the header line
            sequences[uniprot_id] = sequence
        else:
            print(
                f"Failed to retrieve sequence for UniProt ID {uniprot_id}. \
                    HTTP Status: {response.status_code}"
            )

    return sequences

## Cleaning up data frame

### Uniprot Codes

#### Drop rows with missing values

In [192]:
# check for missing values in uniprotcode

len(df[df['uniprotcode'].isna()])

334

In [193]:
# remove any rows where uniprotcode is na
df = df.dropna(subset = ['uniprotcode'])

In [194]:
len(df[df['uniprotcode'].isna()])

0

#### Identify duplicates

In [195]:
# check for any duplicated rows

duplicates = df.duplicated()
duplicates.sum()

0

In [196]:
len(df[df['uniprotcode'].duplicated()])

3812

In [197]:
duplicate_ids = df[df['uniprotcode'].duplicated(keep=False)]
differences = duplicate_ids.groupby('uniprotcode').apply(lambda group: group.drop(columns=['uniprotcode']))

In [198]:
differences

id  ordering            family_name_cache  \
uniprotcode                                                     
3SA1_NAJOX  674    680    7367.0           Snake venom toxins   
            3480  3556    7371.0           Snake venom toxins   
            3551  3628    7370.0           Snake venom toxins   
3SI1A_DENAN 3344  3420    7357.0           Snake venom toxins   
            3912  3992    7384.0           Snake venom toxins   
...                ...       ...                          ...   
ZNT8_HUMAN  5323  5456    4328.0  Bacterial zinc transporters   
ZNT8_XENTR  8478  9927    4341.0  Bacterial zinc transporters   
            8479  9928    4340.0  Bacterial zinc transporters   
ZNTA_SHISS  2442  2510    1834.0                     P-ATPase   
            2443  2511    1833.0                     P-ATPase   

                       species_name_cache membrane_name_cache  \
uniprotcode                                                     
3SA1_NAJOX  674               Naja oxiana            Secreted   
            3480              Naja oxiana            Secreted   
            3551              Naja oxiana            Secreted   
3SI1A_DENAN 3344  Dendroaspis angusticeps            Secreted   
            3912  Dendroaspis angusticeps            Secreted   
...                                   ...                 ...   
ZNT8_HUMAN  5323             Homo sapiens     Eykaryo. plasma   
ZNT8_XENTR  8478       Xenopus tropicalis             Vesicle   
            8479       Xenopus tropicalis             Vesicle   
ZNTA_SHISS  2442          Shigella sonnei     Gram-neg. inner   
            2443          Shigella sonnei     Gram-neg. inner   

                                                         name  description  \
uniprotcode                                                                  
3SA1_NAJOX  674                                   Cytotoxin-1          NaN   
            3480                      Cytotoxin-1, minor form          NaN   
            3551                     Cytotoxin-1, in micelles          NaN   
3SI1A_DENAN 3344                      Ancestral Mamba toxin 1          NaN   
            3912                                  Toxin AdTx1          NaN   
...                                                       ...          ...   
ZNT8_HUMAN  5323              Zinc transporter 8, structure 1          NaN   
ZNT8_XENTR  8478                               ZnT8 with zinc          NaN   
            8479                             ZnT8 at a low pH          NaN   
ZNTA_SHISS  2442    Zinc-transporting ATPase, ZntA, E2P state          NaN   
            2443  Zinc-transporting ATPase, ZntA, E2.PI state          NaN   

                 comments    pdbid resolution  ... family_id  superfamily_id  \
uniprotcode                                    ...                             
3SA1_NAJOX  674       NaN  ="1rl5"        NMR  ...        66              53   
            3480      NaN  ="5lue"        NMR  ...        66              53   
            3551      NaN  ="5nq4"        NMR  ...        66              53   
3SI1A_DENAN 3344      NaN  ="5mg9"        1.8  ...        66              53   
            3912      NaN  ="4iye"       1.95  ...        66              53   
...                   ...      ...        ...  ...       ...             ...   
ZNT8_HUMAN  5323      NaN  ="6xpe"     4.1 EM  ...       260             183   
ZNT8_XENTR  8478      NaN  ="7y5g"    3.85 EM  ...       260             183   
            8479      NaN  ="7y5h"    3.72 EM  ...       260             183   
ZNTA_SHISS  2442      NaN  ="4umv"        3.2  ...        30              22   
            2443      NaN  ="4umw"        2.7  ...        30              22   

                  classtype_id  type_id  secondary_representations_count  \
uniprotcode                                                                
3SA1_NAJOX  674              3        2                                3   
            3480             3        2     

In [199]:
df.columns

Index(['id', 'ordering', 'family_name_cache', 'species_name_cache',
       'membrane_name_cache', 'name', 'description', 'comments', 'pdbid',
       'resolution', 'topology_subunit', 'topology_show_in', 'thickness',
       'thicknesserror', 'subunit_segments', 'tilt', 'tilterror', 'gibbs',
       'tau', 'verification', 'membrane_id', 'species_id', 'family_id',
       'superfamily_id', 'classtype_id', 'type_id',
       'secondary_representations_count', 'structure_subunits_count',
       'citations_count', 'created_at', 'updated_at', 'uniprotcode',
       'interpro'],
      dtype='object')

In [200]:
# check how many duplicates exist if dropping specific columns
col_to_drop = ['updated_at', 'created_at', 'citations_count', 'description', 'comments', 'id', 'ordering', 'name', 'pdbid', 'resolution', 'tilt', 'tilterror',
               'gibbs', 'tau', 'thickness', 'thicknesserror', 'secondary_representations_count', 'structure_subunits_count', 'topology_subunit', 'subunit_segments',
               'topology_show_in', 'membrane_id', 'verification', 'interpro']

df_smaller = df.drop(columns = col_to_drop)
df_smaller.reset_index(drop=True)
df_smaller.head()

,family_name_cache,species_name_cache,membrane_name_cache,species_id,family_id,superfamily_id,classtype_id,type_id,uniprotcode
0,OmpA family,Escherichia coli,Gram-neg. outer,9,34,26,2,1,OMPA_ECOLI
1,Enterobacterial Ail/Lom protein,Escherichia coli,Gram-neg. outer,9,355,26,2,1,OMPX_ECOLI
2,Opacity porins,Neisseria meningitidis,Gram-neg. outer,24,337,235,2,1,Q9RP17_NEIME
3,Influenza virus matrix protein 2,Influenza A virus,Viral,51,263,185,11,1,M2_I97A1
4,"OM protease omptin, OMPT",Yersinia pestis,Gram-neg. outer,299,36,27,2,1,COLY_YERPE


In [201]:
df_smaller.columns

Index(['family_name_cache', 'species_name_cache', 'membrane_name_cache',
       'species_id', 'family_id', 'superfamily_id', 'classtype_id', 'type_id',
       'uniprotcode'],
      dtype='object')

In [202]:
len(df_smaller[df_smaller['uniprotcode'].duplicated()])

3812

In [203]:
# merge duplicated rows

df_smaller = df_smaller.drop_duplicates()

In [204]:
len(df_smaller[df_smaller['uniprotcode'].duplicated()])

98

In [205]:
duplicate_ids_smaller = df_smaller[df_smaller['uniprotcode'].duplicated(keep=False)]
differences_smaller = duplicate_ids_smaller.groupby('uniprotcode').apply(lambda group: group.drop(columns=['uniprotcode']))

In [206]:
differences_smaller

family_name_cache  \
uniprotcode                                                                                           
A0A1S1YN08_9BACI                                   7948                     Monoacylglycerol lipase   
                                                   7949                     Monoacylglycerol lipase   
A0A286ZNN4_PIG A0A287AAR5_PIG A0A287AE88_PIG A0... 6978  H+ or Na+ translocating NADH dehydrogenase   
                                                   7008  H+ or Na+ translocating NADH dehydrogenase   
A3DCU1_ACET2                                       2794                       Lipid exporter family   
...                                                                                             ...   
VEMP_SARS                                          3739                       Coronavirus E protein   
VPR_HV1B9                                          1324                 Peptides of viral protein R   
                                                   1325                Viral protein R (VPR) family   
YEJM_ECOLI                                         3091                                   Sulfatase   
                                                   5321                                   Sulfatase   

                                                                          species_name_cache  \
uniprotcode                                                                                    
A0A1S1YN08_9BACI                                   7948         Cytobacillus oceanisediminis   
                                                   7949                      Geobacillus sp.   
A0A286ZNN4_PIG A0A287AAR5_PIG A0A287AE88_PIG A0... 6978                           Sus scrofa   
                                                   7008                           Sus scrofa   
A3DCU1_ACET2                                       2794     Hungateiclostridium thermocellum   
...                                                                                      ...   
VEMP_SARS                                          3739                             SARS-CoV   
VPR_HV1B9                                          1324  Human immunodeficiency virus type 1   
                                                   1325  Human immunodeficiency virus type 1   
YEJM_ECOLI                                         3091               Salmonella typhimurium   
                                                   5321                     Escherichia coli   

                                                        membrane_name_cache  \
uniprotcode                                                                   
A0A1S1YN08_9BACI                                   7948            Secreted   
                                                   7949            Secreted   
A0A286ZNN4_PIG A0A287AAR5_PIG A0A287AE88_PIG A0... 6978     Mitochon. inner   
                                                   7008     Gram-neg. inner   
A3DCU1_ACET2                                       2794     Gram-pos. inner   
...                                                                     ...   
VEMP_SARS                                          3739            Endosome   
VPR_HV1B9                                          1324     Eykaryo. plasma   
                                                   1325     Eykaryo. plasma   
YEJM_ECOLI                                         3091     Gram-neg. inner   
                                                   5321     Gram-neg. inner   

                                                         species_id  \
uniprotcode                                                           
A0A1S1YN08_9BACI                                   7948        1066   
                                                   7949         544   
A0A286ZNN4_PIG A0A287AAR5_PIG A0A287AE88_PIG A0... 6978          56   
                                                   7008          56   
A3DCU1_ACET2                           

In [207]:
df_smaller.shape

(4867, 9)

In [208]:
# drop rows for which still duplicates exist
df_smaller = df_smaller[~df_smaller['uniprotcode'].duplicated(keep=False)]
len(df_smaller[df_smaller['uniprotcode'].duplicated()]), df_smaller.shape

(0, (4679, 9))

#### Deal with rows with multiple protein ids

In [209]:
# find rows whith multiple protein ids

df_smaller[df_smaller['uniprotcode'].str.contains(' ')]

,family_name_cache,species_name_cache,membrane_name_cache,species_id,family_id,superfamily_id,classtype_id,type_id,uniprotcode
29,Chloride transporter,Escherichia coli,Gram-neg. inner,9,18,10,1,1,CLCA_ECOLI HVM38_MOUSE IGKC_MOUSE
32,Methane monooxygenase,Methylococcus capsulatus,Gram-neg. inner,19,31,23,1,1,O05111_METCP PMOA_METCA PMOB_METCA
41,FecCD transport family,Escherichia coli,Gram-neg. inner,9,25,17,1,1,BTUC_ECOLI BTUD_ECOLI
42,Light-harvesting complexes from bacteria,Rhodopseudomonas acidophila,Thylakoid,32,1,2,1,1,LHA4_RHOAC LHB5_RHOAC
43,Light-harvesting complexes from bacteria,Rhodopseudomonas acidophila,Thylakoid,32,1,2,1,1,LHA1_RHOAC LHB1_RHOAC
...,...,...,...,...,...,...,...,...,...
8901,"G-protein coupled receptors, family A",Homo sapiens,Eykaryo. plasma,14,14,6,1,1,C3AR_HUMAN GBB1_HUMAN GBG2_HUMAN GNAI1_HUMAN
8902,"G-protein coupled receptors, family A",Homo sapiens,Eykaryo. plasma,14,14,6,1,1,C3AR_HUMAN CO3_HUMAN GBB1_RAT GBG2_BOVIN GNAI1...
8903,"G-protein coupled receptors, family A",Homo sapiens,Eykaryo. plasma,14,14,6,1,1,C3AR_HUMAN GBB1_RAT GBG2_BOVIN GNAI1_HUMAN
8906,"G-protein coupled receptors, family A",Mus musculus,Eykaryo. plasma,52,14,6,1,1,GBB1_HUMAN GBG2_HUMAN TAAR9_MOUSE


In [210]:
# remove these rows as contain multiple sequences

df_smaller = df_smaller[~df_smaller['uniprotcode'].str.contains(' ')]

In [211]:
def write_fasta(fp, sequences):
    """
    Writes protein sequences to individual FASTA files in a shared directory.

    Args:
        fp (str): Directory path to store the FASTA files.
        sequences (dict): A dictionary where keys are UniProt IDs and values are protein sequences.
    """
    # Ensure the directory exists
    os.makedirs(fp, exist_ok=True)
    
    for uniprot_id, sequence in sequences.items():
        uniprot_id = uniprot_id.strip()
        uniprot_id = uniprot_id.replace(".", "")
        # Define the file path
        file_path = os.path.join(fp, f"{uniprot_id}.fasta")
        
        # Skip writing if the file already exists
        if os.path.exists(file_path):
            # print(f"File for {uniprot_id} already exists. Skipping.")
            continue
        
        # Write the sequence to the file
        with open(file_path, "w") as f:
            f.write(f">{uniprot_id}\n")
            f.write(f"{sequence}\n") 

In [212]:
def batch_process_uniprot(df, batch_size=10):
    """
    Processes UniProt sequences in batches to avoid too many requests at once.

    Args:
        df (DataFrame): DataFrame containing UniProt IDs.
        batch_size (int): Number of sequences to process per batch.
    """
    # Iterate through the DataFrame in batches
    for i in range(0, df.shape[0], batch_size):
        batch_uniprot_ids = df['uniprotcode'].iloc[i:i+batch_size]
        uniprot_sequences = get_sequences_from_uniprot(batch_uniprot_ids)
        
        # Write the sequences to individual FASTA files
        write_fasta("protein_sequences", uniprot_sequences)

In [ ]:
batch_process_uniprot(df_smaller, batch_size=10)

In [213]:
def read_fasta_to_dict(directory):
    """
    Reads all FASTA files in a directory and returns a dictionary of sequences.

    Args:
        directory (str): Path to the directory containing the FASTA files.

    Returns:
        dict: A dictionary where keys are UniProt IDs and values are protein sequences.
    """
    sequences = {}
    
    # Iterate over all files in the directory
    for filename in os.listdir(directory):
        if filename.endswith('.fasta'):
            # Extract UniProt ID from the file name (without extension)
            uniprot_id = filename.replace('.fasta', '')
            
            # Read the sequence from the file
            with open(os.path.join(directory, filename), 'r') as f:
                # Skip the header line
                header = f.readline()
                # Read the sequence (assume it's all in one line)
                sequence = f.read().replace('\n', '')  # Remove any newlines from the sequence
                sequences[uniprot_id] = sequence

    return sequences

In [214]:
sequences = read_fasta_to_dict("protein_sequences")

In [215]:
len(sequences)

3071

In [216]:
len([seq_id for seq_id, seq in sequences.items() if len(seq) == 0])

69

In [217]:
# remove sequences where sequence length is 0 

sequences = {seq_id: seq for seq_id, seq in sequences.items() if len(seq) != 0}

In [218]:
len(sequences)

3002

In [219]:
# find proteins for which sequence hasn't been retriedved yet

proteins = set(df_smaller['uniprotcode'])
proteins_retrieved = set(sequences.keys())

In [220]:
proteins - proteins_retrieved
# could not find the sequences by manually entering them to uniprot

{'A0A087IFK6_VIBVL',
 'A0A0A1RKH5_9ENTR',
 'A0A0C7CQY7_PSEAE',
 'A0A0D6H8R3_ALCXX',
 'A0A0E4HLL9',
 'A0A0F5EN03',
 'A0A0F7VRL1_9ACTN',
 'A0A0G3SCW3_KLEOX',
 'A0A0H3LM39_BORBR',
 'A0A0P0QBS3_SERMA',
 'A0A0S4MEX1_STACP',
 'A0A0T9S5R5_YEREN',
 'A0A0U2WJJ9_9BURK',
 'A0A0U4VTN7_9PSED',
 'A0A0X8F058_PSEFR',
 'A0A1B4TSD3',
 'A0A1D5PBN0_CHICK',
 'A0A1D5PS57_CHICK',
 'A0A1L1RNG8_CHICK',
 'A0A1S0WIC1_KLEP7',
 'A0A1W0WWK2_HYPDU',
 'A0A1Z4FUT4_9CYAN',
 'A0A241YIJ9_ACIBA',
 'A0A2A1YWE9',
 'A0A2P1GRC5_9HIV1',
 'A0A2S3SYP4',
 'A0A2S6F4N3_LEGPH',
 'A0A381AKI5_BREDI',
 'A0A384E130_9METZ',
 'A0A3A0FWV3_9PROT',
 'A0A542C9I8_SERMA',
 'A0A657M1C3_STAHO',
 'A0A6A5PUS7_YEASX',
 'A0A6P3HVI0_BISBI',
 'A0A7Z7L1X9_ECOLI',
 'A0A857SHB2_PROVU',
 'A1CWT6_NEOFI',
 'A1YNH3_9HIV1',
 'A2VST2_9BURK',
 'A3T2G5_SULSN',
 'A4VW80_STRSY',
 'A7LVT2_BACO1',
 'A9JTM7_XENTR',
 'AFTA_MYCT',
 'B0V4F5_ACIBY',
 'B0V9K7_ACIBY',
 'B1P2T4_HORVV',
 'B1XBJ1',
 'B1XBK5',
 'B4EZY7_PROMH',
 'B4UQM4_9INFB',
 'B5I0A0_9ACTN',
 'B6I756_ECOSE',


In [221]:
# subset smaller_df for which protein sequences where retrieved

df_final = df_smaller[df_smaller['uniprotcode'].isin(proteins_retrieved)]
df_final.shape

(3002, 9)

In [222]:
df_final['sequence_length'] = df_final['uniprotcode'].apply(lambda x: len(sequences.get(x, '')))
df_final.head()

/tmp/ipykernel_212295/2767154440.py:1: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



,family_name_cache,species_name_cache,membrane_name_cache,species_id,family_id,superfamily_id,classtype_id,type_id,uniprotcode,sequence_length
0,OmpA family,Escherichia coli,Gram-neg. outer,9,34,26,2,1,OMPA_ECOLI,346
1,Enterobacterial Ail/Lom protein,Escherichia coli,Gram-neg. outer,9,355,26,2,1,OMPX_ECOLI,171
2,Opacity porins,Neisseria meningitidis,Gram-neg. outer,24,337,235,2,1,Q9RP17_NEIME,174
3,Influenza virus matrix protein 2,Influenza A virus,Viral,51,263,185,11,1,M2_I97A1,97
4,"OM protease omptin, OMPT",Yersinia pestis,Gram-neg. outer,299,36,27,2,1,COLY_YERPE,312


In [223]:
df_final['sequence_length'].min(), df_final['sequence_length'].max()

(8, 8081)

In [224]:
# plot distribution of sequence length
fig = px.histogram(df_final, x = "sequence_length")
fig.show()

In [225]:
bins = [0, 100, 300, 500, 1000, 2000, float('inf')]
bin_labels = ['0-100', '100-300', '300-500', '500-1000', '1000-2000', '>2000']
df_final['sequence_length_bin'] = pd.cut(df_final['sequence_length'], bins=bins, labels=bin_labels, right=False)
df_final.head()

/tmp/ipykernel_212295/2153194639.py:3: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



,family_name_cache,species_name_cache,membrane_name_cache,species_id,family_id,superfamily_id,classtype_id,type_id,uniprotcode,sequence_length,sequence_length_bin
0,OmpA family,Escherichia coli,Gram-neg. outer,9,34,26,2,1,OMPA_ECOLI,346,300-500
1,Enterobacterial Ail/Lom protein,Escherichia coli,Gram-neg. outer,9,355,26,2,1,OMPX_ECOLI,171,100-300
2,Opacity porins,Neisseria meningitidis,Gram-neg. outer,24,337,235,2,1,Q9RP17_NEIME,174,100-300
3,Influenza virus matrix protein 2,Influenza A virus,Viral,51,263,185,11,1,M2_I97A1,97,0-100
4,"OM protease omptin, OMPT",Yersinia pestis,Gram-neg. outer,299,36,27,2,1,COLY_YERPE,312,300-500


In [226]:
def add_prefix_to_categories(df, columns):
    for col in columns:
        prefix = ''
        if col == 'species_id':
            prefix = 's'
        elif col == 'family_id':
            prefix = 'f'
        elif col == 'superfamily_id':
            prefix = 'sf'
        elif col == 'classtype_id':
            prefix = 'c'
        elif col == 'type_id':
            prefix = 't'
        
        # Add prefix to the values in the column
        df[col] = df[col].apply(lambda x: f"{prefix}{x}")
    
    return df


In [227]:
df_final  = add_prefix_to_categories(df_final, ['species_id', 'family_id', 'superfamily_id', 'classtype_id', 'type_id'])

/tmp/ipykernel_212295/4053373897.py:16: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

/tmp/ipykernel_212295/4053373897.py:16: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

/tmp/ipykernel_212295/4053373897.py:16: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

/tmp/ipyke

In [255]:
def move_column_to_front(df, column_name):
    """
    Moves a specific column to the first position in the DataFrame.

    Args:
        df (pd.DataFrame): The DataFrame to modify.
        column_name (str): The column to move to the front.

    Returns:
        pd.DataFrame: A new DataFrame with the column moved to the front.
    """
    # Ensure the column exists in the DataFrame
    if column_name in df.columns:
        # Reorder the columns
        columns = [column_name] + [col for col in df.columns if col != column_name]
        return df[columns]
    else:
        raise ValueError(f"Column '{column_name}' not found in DataFrame.")

In [257]:
df_final = move_column_to_front(df_final, "uniprotcode")

In [258]:
df_final.head()

,uniprotcode,family_name_cache,species_name_cache,membrane_name_cache,species_id,family_id,superfamily_id,classtype_id,type_id,sequence_length,sequence_length_bin
0,OMPA_ECOLI,OmpA family,Escherichia coli,Gram-neg. outer,s9,f34,sf26,c2,t1,346,300-500
1,OMPX_ECOLI,Enterobacterial Ail/Lom protein,Escherichia coli,Gram-neg. outer,s9,f355,sf26,c2,t1,171,100-300
2,Q9RP17_NEIME,Opacity porins,Neisseria meningitidis,Gram-neg. outer,s24,f337,sf235,c2,t1,174,100-300
3,M2_I97A1,Influenza virus matrix protein 2,Influenza A virus,Viral,s51,f263,sf185,c11,t1,97,0-100
4,COLY_YERPE,"OM protease omptin, OMPT",Yersinia pestis,Gram-neg. outer,s299,f36,sf27,c2,t1,312,300-500


In [230]:
def write_all_sequences_to_fasta(sequences, file_path):
    """
    Writes all protein sequences from a dictionary to a single FASTA file.

    Args:
        sequences (dict): A dictionary where keys are UniProt IDs and values are protein sequences.
        file_path (str): The path to the output FASTA file.
    """
    with open(file_path, "w") as f:
        for uniprot_id, sequence in sequences.items():
            f.write(f">{uniprot_id}\n")
            f.write(f"{sequence}\n")

In [254]:
len(sequences), len(df_final)

(3002, 3002)

In [259]:
# save data

write_all_sequences_to_fasta(sequences, "opm.fasta")

df_final.to_csv("opm_metadata.csv", index = False)

In [266]:
# Step 1: Count the number of unique families for each superfamily
superfamily_family_counts = df_final.groupby('superfamily_id')['family_id'].nunique()

# Step 2: Set a threshold (e.g., superfamilies with more than 1 unique family)
threshold = 17
superfamilies_with_diff_families = superfamily_family_counts[superfamily_family_counts > threshold]

# Step 3: Filter the original dataframe to include only superfamilies with multiple families
df_filtered = df_final[df_final['superfamily_id'].isin(superfamilies_with_diff_families.index)]

# Step 4: Create a pivot table (contingency table) with family_id and superfamily_id
pivot_table = pd.crosstab(df_filtered['family_id'], df_filtered['superfamily_id'])

# Step 5: Create a heatmap using plotly express (swapped axes)
fig = px.imshow(
    pivot_table,
    labels={'x': 'Superfamily ID', 'y': 'Family ID', 'color': 'Count of Samples'},
    color_continuous_scale='Blues',  # Color scale
    title="Families vs Superfamilies Heatmap"
)

# Show the plot
fig.show()

In [282]:
# check how many proteins are only in one family

def count_single_entry_families(df, family_col, x):
    """
    Counts how many families have only one entry in the DataFrame.

    Args:
        df (pd.DataFrame): DataFrame containing protein data.
        family_col (str): Column name for family IDs.

    Returns:
        int: Number of families with only one entry.
    """
    # Count occurrences of each family
    family_counts = df[family_col].value_counts()

    # Filter families with only one entry
    single_entry_families = family_counts[family_counts < x]

    return len(single_entry_families)

num_single_entry_families = count_single_entry_families(df_final, family_col="family_id", x = 10)
num_single_entry_families


865

In [286]:
results = []

# Iterate over x from 0 to 30
for x in range(0, 31):
    num_proteins = count_single_entry_families(df_final, "family_id", x)
    results.append({"Threshold": x, "Proteins": num_proteins})

# Convert results to a DataFrame
results_df = pd.DataFrame(results)

# Plot using Plotly Express
fig = px.line(
    results_df,
    x="Threshold",
    y="Proteins",
    title="Number of Proteins in Families with Size Less Than x",
    labels={"Threshold": "Family Size Threshold (x)", "Proteins": "Number of Proteins"},
    template="plotly_white",
)
fig.update_layout(font=dict(family="Arial", size=12, color="black"))
fig.show()

In [283]:
def remove_small_families(df, family_col, min_entries):
    """
    Removes all rows belonging to families with fewer than `min_entries` entries.

    Args:
        df (pd.DataFrame): DataFrame containing protein data.
        family_col (str): Column name for family IDs.
        min_entries (int): Minimum number of entries required for a family to be retained.

    Returns:
        pd.DataFrame: Filtered DataFrame.
    """
    # Count occurrences of each family
    family_counts = df[family_col].value_counts()

    # Identify families with at least `min_entries` entries
    valid_families = family_counts[family_counts >= min_entries].index

    # Filter DataFrame to keep only valid families
    filtered_df = df[df[family_col].isin(valid_families)]

    return filtered_df

In [287]:
filtered_df = remove_small_families(df_final, family_col="family_id", min_entries=10)

In [290]:
filtered_df.shape, filtered_df['family_id'].nunique(), filtered_df['superfamily_id'].nunique()

((1072, 11), 51, 39)

In [292]:
family_counts = filtered_df["family_id"].value_counts().reset_index()
family_counts.columns = ["Family ID", "Protein Count"]

# Plot the bar chart
fig = px.bar(
    family_counts,
    x="Family ID",
    y="Protein Count",
    title="Protein Count per Family",
    labels={"Family ID": "Family ID", "Protein Count": "Number of Proteins"},
    template="plotly_white",
)

# Customize layout
fig.update_layout(
    font=dict(family="Arial", size=12, color="black"),
    xaxis=dict(tickangle=45),
    showlegend=False,
)

# Display the plot
fig.show()

In [296]:
filtered_df['family_id'].value_counts()

family_id
f52      57
f107     56
f154     52
f13      42
f62      40
f137     40
f60      39
f682     36
f14      35
f42      30
f66      27
f71      26
f67      26
f212     25
f158     25
f147     24
f15      23
f83      23
f72      21
f184     21
f54      18
f259     18
f148     18
f183     17
f219     16
f70      16
f1156    15
f104     15
f74      15
f153     14
f785     14
f333     14
f656     13
f852     13
f63      13
f264     13
f197     13
f92      13
f99      12
f401     11
f263     11
f123     11
f157     11
f194     10
f186     10
f178     10
f172     10
f439     10
f19      10
f812     10
f215     10
Name: count, dtype: int64

In [293]:
family_counts = filtered_df["superfamily_id"].value_counts().reset_index()
family_counts.columns = ["Super Family ID", "Protein Count"]

# Plot the bar chart
fig = px.bar(
    family_counts,
    x="Super Family ID",
    y="Protein Count",
    title="Protein Count per Superfamily",
    labels={"Family ID": "Super Family ID", "Protein Count": "Number of Proteins"},
    template="plotly_white",
)

# Customize layout
fig.update_layout(
    font=dict(family="Arial", size=12, color="black"),
    xaxis=dict(tickangle=45),
    showlegend=False,
)

# Display the plot
fig.show()

In [297]:
filtered_df.to_csv("opm_subset_metadata.csv", index = False)

In [ ]:
fig = px.histogram(filtered_df, x="")

In [278]:
df_final.shape

(3002, 11)

In [269]:
def subset_proteins_by_superfamilies(df, superfamily_col, family_col, limit_families=15, select_superfamilies=10):
    """
    Subsets proteins to include only those from a limited number of superfamilies
    where each superfamily has at most a given number of families.

    Args:
        df (pd.DataFrame): DataFrame containing protein information.
        superfamily_col (str): Column name for superfamily IDs.
        family_col (str): Column name for family IDs.
        limit_families (int): Maximum number of families per superfamily.
        select_superfamilies (int): Number of superfamilies to select.

    Returns:
        pd.DataFrame: Subset of the original DataFrame.
    """
    # Count families in each superfamily
    family_counts = df.groupby(superfamily_col)[family_col].nunique()
    
    # Filter superfamilies with <= limit_families
    eligible_superfamilies = family_counts[family_counts <= limit_families].index
    
    # Select up to `select_superfamilies` superfamilies
    selected_superfamilies = eligible_superfamilies[:select_superfamilies]
    
    # Subset DataFrame
    subset_df = df[df[superfamily_col].isin(selected_superfamilies)]
    return subset_df

# Example usage
# Assuming your DataFrame is named df_proteins
subset_df = subset_proteins_by_superfamilies(
    df=df_filtered,
    superfamily_col="superfamily_id",
    family_col="family_id",
    limit_families=15,
    select_superfamilies=3
)
print(subset_df)

Empty DataFrame
Columns: [uniprotcode, family_name_cache, species_name_cache, membrane_name_cache, species_id, family_id, superfamily_id, classtype_id, type_id, sequence_length, sequence_length_bin]
Index: []


In [273]:
eligible_superfamilies = family_counts[family_counts <= 15].index

In [275]:
len(eligible_superfamilies)

890